In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
from dateutil.parser import parse
import copy

#시각화
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import *
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss

##알고리즘 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [6]:
# 다양한 오차 측정 지표를 확인하기 위한 함수 정의

from sklearn.metrics import *

def get_clf_eval(y_test, pred=None, pred_proba=None): 
    confusion = confusion_matrix(y_test, pred) 
    accuracy = accuracy_score(y_test, pred) 
    precision = precision_score(y_test, pred , average= 'weighted') 
    recall = recall_score(y_test, pred , average= 'weighted') 
    f1 = f1_score(y_test, pred , average= 'weighted') 
    
#     print('정확도(accuracy): {0:.4f}, 정밀도(precision): {1:.4f}, 재현율(recall): {2:.4f}, f1_score: {3:.4f}'.format(accuracy, precision, recall, f1))
    
#     confusion = confusion_matrix(y_test, pred) 
#     accuracy = accuracy_score(y_test, pred) 
#     precision = precision_score(y_test, pred , average= 'macro') 
#     recall = recall_score(y_test, pred , average= 'macro') 
#     f1 = f1_score(y_test, pred , average= 'macro') 
    
#     print('정확도(accuracy): {0:.4f}, 정밀도(precision): {1:.4f}, 재현율(recall): {2:.4f}, f1_score: {3:.4f}'.format(accuracy, precision, recall, f1))
    
    return [accuracy, precision, recall, f1]

In [3]:
def Cut(Series , cuts):
    "구분할 iter가능한 변수와 구분 기준을 입력받으면 구분 기준 앞에서 부터 1로 구분해서 return 함"
    Cuts = copy.deepcopy(cuts)
    Cuts.append(np.inf)
    Cuts.insert(0,-1*np.inf)
    R = len(Cuts)
    label = range(1,R)
    return pd.cut(Series, Cuts, labels = label)

# 데이터 불러오기

In [73]:
df_1_3 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_After_IPO_1.csv")
df_3_6 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_After_IPO_3.csv")
df_0 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_Before_IPO.csv")

In [74]:
df_0_1 = df_0.drop(['공모가 대비 3개월 수익률','공모가 대비 6개월 수익률'],axis =1)
df_0_3 = df_0.drop(['공모가 대비 1개월 수익률','공모가 대비 6개월 수익률'],axis =1)
df_0_6 = df_0.drop(['공모가 대비 1개월 수익률','공모가 대비 3개월 수익률'],axis =1)

# 공통 전처리 및 랜덤포레스트

In [75]:
def process_final(df,y_name):
    
    ## 기본 전처리
    df['상장일'] = pd.to_datetime(df['상장일'])
    df = df.set_index(['상장일'])
    df = df.drop(['종목명','공모 시가총액'],axis = 1) ## 나중에 카테고리 진행할려면 남겨줄 것
    if y_name == '공모가 대비 6개월 수익률':
        df['Cat'] = Cut(df[y_name],[-0.4, -0.2, 0.2, 0.4])
       
        
    else :
        df['Cat'] = Cut(df[y_name],[-0.2, -0.1, 0.1, 0.2])
       
    
    ## train 3년 test 1년으로 총 기간을 3개월 이동으로 36개 구간분할
    train_list = []
    test_list = []
    train_start_date = '2009-04-01' ## 기한은 나중에 변경할수도
    test_start_date = parse(str(train_start_date)).date() + relativedelta(years =3)
    train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
    test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1)

    while True:

        train_list.append(df[train_start_date : train_end_date])
        test_list.append(df[test_start_date : test_end_date])

        train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
        test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
        train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
        test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)

        if str(train_start_date) == '2018-04-01':
            break
            
    y_data = []
    score_list = []
    feature_list = []

    for i in range(0,len(train_list)):
        X_train = train_list[i].drop([y_name,'Cat'],axis =1)
        y_train = train_list[i]['Cat']
        X_test = test_list[i].drop([y_name,'Cat'],axis =1)
        y_test = test_list[i]['Cat']

            # 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
        clf = RandomForestClassifier(random_state=0 , max_depth = 10 , min_samples_leaf = 4 ,\
                                    min_samples_split =  4 , n_estimators = 400)

        clf.fit(X_train , y_train)
        train_pred = clf.predict(X_train)
        test_pred = clf.predict(X_test)

        # 성과 평가
        train_res = get_clf_eval(y_train, train_pred)
        test_res = get_clf_eval(y_test,test_pred)
        res = train_res + test_res


        #feature_importance
        feature_importance = clf.feature_importances_


        pred_value = pd.Series(test_pred,index = y_test.index)
        per = test_list[i][y_name]
        y_testdata = pd.concat([per,y_test,pred_value] , axis = 1)
        y_testdata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']

        score_list.append(res)
        feature_list.append(feature_importance)
        y_data.append(y_testdata)   
    df_score = pd.DataFrame(score_list,columns = ["정확도","정밀도","재현율","f1_score"]*2).T 
    df_score.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)

    df_feature = pd.DataFrame(feature_list,columns = X_train.columns).T 
    df_feature.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)
        
    return df_score , df_feature , y_data

In [76]:
df_score_0_1,df_feature_0_1,y_data_0_1 = process_final(df_0_1,'공모가 대비 1개월 수익률')

In [36]:
df_score_0_1

,model_set_0,model_set_1,model_set_2,model_set_3,model_set_4,model_set_5,model_set_6,model_set_7,model_set_8,model_set_9,...,model_set_26,model_set_27,model_set_28,model_set_29,model_set_30,model_set_31,model_set_32,model_set_33,model_set_34,model_set_35
정확도,0.903226,0.905109,0.904412,0.894737,0.905512,0.925000,0.918182,0.900000,0.820000,0.821053,...,0.816216,0.821839,0.833333,0.823864,0.825397,0.823529,0.849462,0.847458,0.797753,0.800000
정밀도,0.913045,0.912863,0.912829,0.903943,0.914458,0.929611,0.923801,0.913027,0.865377,0.870923,...,0.848081,0.854445,0.861947,0.851887,0.859104,0.861582,0.871218,0.875091,0.838273,0.841604
재현율,0.903226,0.905109,0.904412,0.894737,0.905512,0.925000,0.918182,0.900000,0.820000,0.821053,...,0.816216,0.821839,0.833333,0.823864,0.825397,0.823529,0.849462,0.847458,0.797753,0.800000
f1_score,0.894355,0.901494,0.889925,0.890662,0.902065,0.923425,0.916746,0.896122,0.813132,0.814518,...,0.797385,0.802423,0.812998,0.809112,0.800245,0.798267,0.829193,0.811329,0.768934,0.761893
정확도,0.454545,0.440000,0.363636,0.277778,0.387097,0.354839,0.432432,0.553191,0.543478,0.526316,...,0.390625,0.437500,0.312500,0.311475,0.359375,0.360656,0.513889,0.647059,0.720930,0.736842
정밀도,0.366883,0.514921,0.458235,0.394741,0.274194,0.258065,0.186998,0.306021,0.295369,0.277008,...,0.345895,0.348843,0.168280,0.200556,0.279545,0.376923,0.476906,0.497011,0.549280,0.542936
재현율,0.454545,0.440000,0.363636,0.277778,0.387097,0.354839,0.432432,0.553191,0.543478,0.526316,...,0.390625,0.437500,0.312500,0.311475,0.359375,0.360656,0.513889,0.647059,0.720930,0.736842
f1_score,0.398601,0.453333,0.352597,0.281312,0.321007,0.298812,0.261091,0.394054,0.382731,0.362976,...,0.321801,0.369052,0.214747,0.233931,0.313889,0.363698,0.493013,0.561375,0.623507,0.625199


In [37]:
df_feature_0_1

,model_set_0,model_set_1,model_set_2,model_set_3,model_set_4,model_set_5,model_set_6,model_set_7,model_set_8,model_set_9,...,model_set_26,model_set_27,model_set_28,model_set_29,model_set_30,model_set_31,model_set_32,model_set_33,model_set_34,model_set_35
매출총이익률,0.085207,0.084091,0.085492,0.107209,0.096871,0.094696,0.080716,0.111761,0.102269,0.094843,...,0.076235,0.078086,0.069879,0.068004,0.066263,0.076006,0.069079,0.071267,0.058594,0.064350
유동비율,0.079238,0.095063,0.087352,0.098819,0.101942,0.091010,0.089332,0.086453,0.089019,0.092576,...,0.076780,0.085476,0.087229,0.096705,0.078674,0.086385,0.079810,0.083365,0.076441,0.066186
부채비율,0.077796,0.090853,0.088029,0.088790,0.095030,0.099366,0.099393,0.083276,0.097158,0.101959,...,0.081098,0.076230,0.076005,0.085012,0.097203,0.100555,0.102926,0.120803,0.107906,0.098870
매출액증가율,0.076601,0.084820,0.087776,0.077193,0.081423,0.073130,0.072237,0.076630,0.072276,0.082591,...,0.097825,0.100064,0.098223,0.080741,0.094418,0.101865,0.095456,0.091908,0.085941,0.087328
IPO_PER,0.081482,0.077852,0.083249,0.082196,0.083109,0.100123,0.097612,0.101227,0.093936,0.097503,...,0.088233,0.072342,0.079336,0.084035,0.099674,0.095843,0.100656,0.089633,0.086652,0.088447
IPO_PBR,0.093719,0.089656,0.082432,0.086857,0.077118,0.080117,0.096721,0.091151,0.087019,0.081186,...,0.156810,0.163484,0.145737,0.151577,0.163408,0.127025,0.128953,0.115416,0.114029,0.095229
IPO_EV/EBITDA,0.077015,0.085678,0.086200,0.081485,0.088918,0.093206,0.092090,0.089336,0.102590,0.113848,...,0.099951,0.099020,0.101339,0.106276,0.114600,0.106716,0.104195,0.098968,0.110088,0.096258
1년전 대비 시장 회전율,0.091981,0.095643,0.086672,0.079336,0.097455,0.076603,0.090811,0.081853,0.074129,0.074353,...,0.073572,0.083060,0.076832,0.077247,0.076387,0.080383,0.082016,0.084584,0.096626,0.113105
1년 전 대비 시장수익률,0.164291,0.095957,0.092251,0.097414,0.093030,0.083614,0.082129,0.081645,0.076221,0.076526,...,0.078061,0.073613,0.082120,0.078402,0.062867,0.073909,0.080765,0.084025,0.073187,0.070908
1년 전 대비 고객예탁금 증감률,0.096868,0.110430,0.135081,0.125662,0.098222,0.091330,0.085883,0.089819,0.102030,0.100875,...,0.083982,0.082860,0.082100,0.076361,0.072635,0.070435,0.075811,0.085194,0.095832,0.114355


In [23]:
y_data_0_1[0]

,실제 y 수익률,실제 y라벨링,예측 y 라벨링
상장일,,,
2012-05-04,-0.122642,2,3
2012-07-06,-0.179412,2,3
2012-07-13,-0.026087,3,3
2012-07-19,-0.032222,3,3
2012-07-25,-0.333333,1,3
2012-07-26,-0.034545,3,3
2012-08-02,0.578571,5,3
2012-09-25,0.450000,5,5
2012-09-27,-0.017333,3,5


In [38]:
df_score_0_3,df_feature_0_3,y_data_0_3 = process_final(df_0_3,'공모가 대비 3개월 수익률')

In [39]:
df_score_0_6,df_feature_0_6,y_data_0_6 = process_final(df_0_6,'공모가 대비 6개월 수익률')

In [40]:
df_score_1_3,df_feature_1_3,y_data_1_3 = process_final(df_1_3,'1_3_수익률')

In [41]:
df_score_3_6,df_feature_3_6,y_data_3_6 = process_final(df_3_6,'3_6_수익률')

In [42]:
df_score_0_1.to_csv("0_1_모델 평가.csv",encoding = 'euc-kr')
df_score_0_3.to_csv("0_3_모델 평가.csv",encoding = 'euc-kr')
df_score_0_6.to_csv("0_6_모델 평가.csv",encoding = 'euc-kr')
df_score_1_3.to_csv("1_3_모델 평가.csv",encoding = 'euc-kr')
df_score_3_6.to_csv("3_6_모델 평가.csv",encoding = 'euc-kr')

In [44]:
df_feature_0_1.to_csv("0_1_feature별 값.csv",encoding = 'euc-kr')
df_feature_0_3.to_csv("0_3_feature별 값.csv",encoding = 'euc-kr')
df_feature_0_6.to_csv("0_6_feature별 값.csv",encoding = 'euc-kr')
df_feature_1_3.to_csv("1_3_feature별 값.csv",encoding = 'euc-kr')
df_feature_3_6.to_csv("3_6_feature별 값.csv",encoding = 'euc-kr')

In [110]:
buy = y_data_0_1[9][y_data_0_1[9]['예측 y 라벨링'] > 3]['실제 y 수익률'].mean()

In [111]:
buy

0.44131470607017537

In [122]:
sell = y_data_0_1[9][y_data_0_1[9]['예측 y 라벨링'] < 3]['실제 y 수익률'].mean()

In [130]:
np.isnan(sell)

True

In [96]:
buy - sell

nan

In [131]:
performance = []
for i in range(0,36):
    buy = y_data_0_1[i][y_data_0_1[i]['예측 y 라벨링'] > 3]['실제 y 수익률'].mean()
    sell = y_data_0_1[i][y_data_0_1[i]['예측 y 라벨링'] < 3]['실제 y 수익률'].mean()
    if np.isnan(sell):
        sell = 0
    performance.append(buy-sell)

In [132]:
performance

[0.04083190216666671,
 0.07938080723809526,
 -0.1655938737142858,
 -0.0960876341470589,
 -0.32778234629166675,
 -0.37160048390909095,
 0.195303629945946,
 0.4892079609361703,
 0.44851698410869567,
 0.44131470607017537,
 0.4811036566935483,
 0.3720128916111112,
 0.3568972239743591,
 0.30067577691935493,
 -2.02291637414,
 0.08975704324,
 0.15160934900000003,
 0.2049561671794872,
 0.1874854034489796,
 0.2283780789655172,
 0.2470610157627118,
 0.2955622280576923,
 0.29090362117021273,
 0.1621721728545455,
 0.1719224592340425,
 0.22125062212121216,
 0.1959024750784314,
 0.3226845115294117,
 -0.35915770387499996,
 -0.23709086685294115,
 0.16857339097272733,
 0.6510102051714287,
 0.13266326235220127,
 0.19607126932258068,
 -1.120637178,
 0.6522706035789474]

In [133]:
avg = sum(performance,0.0)/len(performance)

In [134]:
avg

0.08540591549372804